In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI Vizier优化多个目标

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vizier/gapic-vizier-multi-objective-optimization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vizier/gapic-vizier-multi-objective-optimization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vizier/gapic-vizier-multi-objective-optimization.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了[Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview)的多目标优化。多目标优化涉及同时优化多个目标函数的数学优化问题。

了解更多关于[Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview)的信息。

### 目标

在本教程中，您将学习如何使用`Vertex AI Vizier`来优化多目标研究。

目标是__`最小化`__目标指标：
   ```
   y1 = r*sin(theta)
   ```

并同时__`最大化`__目标指标：
   ```
   y2 = r*cos(theta)
   ```

您将在参数空间中进行评估：

   - __`r`__ 在[0,1]之间，

   - __`theta`__ 在[0, pi/2]之间

### 成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI

了解有关[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)的信息，并使用[定价计算器](https://cloud.google.com/products/calculator/)
根据预估的使用情况生成成本估算。

安装

安装运行此笔记本需要的软件包。

In [ ]:
! pip3 install --quiet google-vizier==0.0.4
! pip3 install --upgrade --quiet google-cloud-aiplatform

只有 Colab: 取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面： [查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 设置地区

**可选**: 更新“REGION”变量以指定要使用的地区。了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的Google云账号

要认证您的Google云账号，请按照您的Jupyter环境的指示操作：

**1. Vertex AI Workbench**
<br>您已经认证。

**2. 本地JupyterLab实例**
<br>取消注释并运行以下代码：

In [ ]:
# ! gcloud auth login

取消注释并运行以下代码：

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

请参阅如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务账户授予云存储权限。

导入库并定义常量

In [ ]:
import datetime

from google.cloud import aiplatform
from google.cloud.aiplatform.vizier import Study, pyvizier

教程

这部分定义了一些参数和实用方法来调用Vertex Vizier APIs。请填写以下信息以开始。

In [ ]:
# These will be automatically filled in.
STUDY_DISPLAY_NAME = "{}_study_{}".format(
    PROJECT_ID.replace("-", ""), datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
)
PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

print("REGION: {}".format(REGION))
print("PARENT: {}".format(PARENT))

### 创建研究配置

以下是一个示例研究配置，构建为一个层次结构的Python字典。已经填写完毕。运行该单元格以配置研究。

In [ ]:
# Parameter Configuration
problem = pyvizier.StudyConfig()
problem.algorithm = pyvizier.Algorithm.RANDOM_SEARCH

# Objective Metrics
problem.metric_information.append(
    pyvizier.MetricInformation(name="y1", goal=pyvizier.ObjectiveMetricGoal.MINIMIZE)
)
problem.metric_information.append(
    pyvizier.MetricInformation(name="y2", goal=pyvizier.ObjectiveMetricGoal.MAXIMIZE)
)

# Defines the parameters configuration.
root = problem.search_space.select_root()
root.add_float_param("r", 0, 1.0, scale_type=pyvizier.ScaleType.LINEAR)
root.add_float_param("theta", 0, 1.57, scale_type=pyvizier.ScaleType.LINEAR)

创造研究

接下来，创造研究，随后您将运行以优化这两个目标。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
study = Study.create_or_load(display_name=STUDY_DISPLAY_NAME, problem=problem)

STUDY_ID = study.name
print("STUDY_ID: {}".format(STUDY_ID))

### 评估度量函数

接下来，定义一些函数来评估这两个客观度量指标。

In [ ]:
import math


# r * sin(theta)
def Metric1Evaluation(r, theta):
    """Evaluate the first metric on the trial."""
    return r * math.sin(theta)


# r * cos(theta)
def Metric2Evaluation(r, theta):
    """Evaluate the second metric on the trial."""
    return r * math.cos(theta)


def CreateMetrics(trial_id, r, theta):
    print(("=========== Start Trial: [{}] =============").format(trial_id))

    # Evaluate both objective metrics for this trial
    y1 = Metric1Evaluation(r, theta)
    y2 = Metric2Evaluation(r, theta)
    print(
        "[r = {}, theta = {}] => y1 = r*sin(theta) = {}, y2 = r*cos(theta) = {}".format(
            r, theta, y1, y2
        )
    )
    measurement = pyvizier.Measurement()
    measurement.metrics["y1"] = y1
    measurement.metrics["y2"] = y2

    # Return the results for this trial
    return measurement

### 设置运行试验的配置参数

__`client_id`__: 请求建议的客户端的标识符。如果多个`SuggestTrialsRequests`具有相同的`client_id`，服务将在试验处于`PENDING`状态时返回相同的建议试验，并在上次建议试验完成后提供新的试验。

__`suggestion_count_per_request`__: 单个请求中请求的建议（试验）数目。

__`max_trial_id_to_stop`__: 在停止之前要探索的试验数量。为了缩短代码运行时间，将其设置为4，因此不要期望收敛。要达到收敛，可能需要大约20次试验（一个好的经验法则是将总维度乘以10）。

In [ ]:
worker_id = "worker1"  # @param {type: 'string'}
suggestion_count_per_request = 3  # @param {type: 'integer'}
max_trial_id_to_stop = 6  # @param {type: 'integer'}

print("worker_id: {}".format(worker_id))
print("suggestion_count_per_request: {}".format(suggestion_count_per_request))
print("max_trial_id_to_stop: {}".format(max_trial_id_to_stop))

### 运行Vertex Vizier试验

运行试验。

In [ ]:
while len(study.trials()) < max_trial_id_to_stop:
    trials = study.suggest(count=suggestion_count_per_request, worker=worker_id)

    for suggested_trial in trials:
        measurement = CreateMetrics(
            suggested_trial.name,
            suggested_trial.parameters["r"].value,
            suggested_trial.parameters["theta"].value,
        )
        suggested_trial.add_measurement(measurement=measurement)
        suggested_trial.complete(measurement=measurement)

列出最佳解决方案

list_optimal_trials返回多目标研究的Pareto最优试验或单目标研究的最佳试验。在这种情况下，如果在之前的步骤中定义了多目标，将返回Pareto最优试验。

In [ ]:
optimal_trials = study.optimal_trials()
print("optimal_trials: {}".format(optimal_trials))

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。您还可以手动删除通过运行以下代码创建的资源。

In [ ]:
study.delete()